In [ ]:
def read_in_chunks(filename):
    all = []
    with open(filename, 'r', encoding='utf-8') as f:
        buffer = []
        for line in f:
            line = line.strip()  # remove whitespace/newline
            if not line:  # skip empty lines
                continue
            buffer.append(line)
            if len(buffer) == 4:
                buffer = []
    return buffer
    

# Example usage
buffer = read_in_chunks("last_hidden.txt")


In [ ]:
len(buffer)

In [ ]:
import json

file_path = "/home/ly/TestEval/data/tgt_paths.jsonl"

data = []
with open(file_path, "r") as file:
    for line in file:
        data.append(json.loads(line))  # Parse each line separately

print(data)


In [ ]:
len(data[2]['sampled_paths'])

In [ ]:
data[2]['sampled_paths']

In [ ]:
data[6]['sampled_condition_paths']

In [ ]:
data[6].keys()

In [ ]:
import json

file_path = "/home/ly/TestEval/predictions/totalcov_deepseek-coder-1.3b-instructformat.jsonl"

data = []
with open(file_path, "r") as file:
    for line in file:
        data.append(json.loads(line))  # Parse each line separately

print(data)


In [ ]:
data[0]

In [ ]:
def get_actual_output(test_code: str):
    local_vars = {}
    
    # Chuyển assert thành return
    modified_code = test_code.replace(
        "assert solution.findMedianSortedArrays(nums1, nums2) == 2",
        "return solution.findMedianSortedArrays(nums1, nums2)"
    )
    
    # Thêm tên hàm để exec định nghĩa hàm nhưng không chạy
    exec(modified_code, local_vars)
    
    # Lấy hàm từ namespace
    test_func = local_vars['test_findMedianSortedArrays']
    
    # Gọi hàm để lấy output
    actual_output = test_func()
    
    return actual_output

# Ví dụ
test_code = '''from tmp_0_3.under_test import Solution
def test_findMedianSortedArrays():
    solution = Solution()
    nums1 = [1, 3]
    nums2 = [2]
    assert solution.findMedianSortedArrays(nums1, nums2) == 2'''

print(get_actual_output(test_code))

In [ ]:
code = "x = 5\ny = 10\nprint(x + y)"
exec(code)


In [ ]:
import re

def get_actual_output(test_code: str):
    local_vars = {}
    # Thay thế mọi assert ... == ... bằng return ...
    def replace_assert(match):
        expr = match.group(1)
        return f"return {expr}"
    modified_code = re.sub(
        r"assert\s+(.+?)\s*==\s*.+", 
        replace_assert, 
        test_code
    )
    exec(modified_code, local_vars)
    # Tìm tên hàm test (giả sử chỉ có một hàm test)
    test_func_name = [k for k in local_vars if k.startswith("test_")][0]
    test_func = local_vars[test_func_name]
    actual_output = test_func()
    return actual_output

# Ví dụ
test_code = '''from tmp_1_3.under_test import Solution\ndef test_isMatch():\n    solution = Solution()\n    assert solution.isMatch('', '') == True\ntest_isMatch()'''

print(get_actual_output(test_code))

In [31]:
import re

def get_actual_output(test_code: str):
    local_vars = {}
    expected = None
    # Chỉ lấy expected từ assert đầu tiên
    m = re.search(r"assert\s+(.+?)\s*==\s*(.+)", test_code)
    if m:
        expected = eval(m.group(2))
    def replace_assert(match):
        expr = match.group(1)
        return f"return {expr}"
    modified_code = re.sub(
        r"assert\s+(.+?)\s*==\s*.+",
        replace_assert,
        test_code
    )
    exec(modified_code, local_vars)
    test_func_name = [k for k in local_vars if k.startswith("test_")][0]
    test_func = local_vars[test_func_name]
    actual_output = test_func()
    
    return actual_output==expected, actual_output, expected

# Ví dụ
test_code = '''from tmp_0_3.under_test import Solution
def test_findMedianSortedArrays():
    solution = Solution()
    nums1 = [1, 3]
    nums2 = [2]
    assert solution.findMedianSortedArrays(nums1, nums2) == 2'''

print(get_actual_output(test_code))

(True, 2, 2)
